In [1]:
%run util.py

/home/jzh/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_folder = '/home/jzh/Waze/'

In [3]:
import json

In [4]:
# Loading PL data
PL_dict = zload(data_folder + 'PL_dict.pkz') 

In [5]:
# PL_dict

In [6]:
mu_1 = PL_dict['PL_AM']  # normal PL for AM period
mu_2 = PL_dict['PL_MD']  # normal PL for MD period
mu_3 = PL_dict['PL_PM']  # normal PL for PM period
mu_4 = PL_dict['PL_NT']  # normal PL for NT period

mu_1 = mu_adjust(mu_1)  # normal PL
mu_01, mu1, mu_11, P1, G_11, H_11, U_11 = ChainGen_(mu_1)

mu_2 = mu_adjust(mu_2)  # normal PL
mu_02, mu2, mu_12, P2, G_12, H_12, U_12 = ChainGen_(mu_2)

mu_3 = mu_adjust(mu_3)  # normal PL
mu_03, mu3, mu_13, P3, G_13, H_13, U_13 = ChainGen_(mu_3)

mu_4 = mu_adjust(mu_4)  # normal PL
mu_04, mu4, mu_14, P4, G_14, H_14, U_14 = ChainGen_(mu_4)

<string>:486: RuntimeWarning: covariance is not positive-semidefinite.


In [7]:
beta = 0.001
G_list = [G_11, G_12, G_13, G_14]
H_list = [H_11, H_12, H_13, H_14]
U_list = [U_11, U_12, U_13, U_14]

eta_wc = {}
eta_Sanov = {}

# Get thresholds for Hoeffding's test corresponding to sample length n  
def etaWC(n):
    return HoeffdingRuleMarkovRobust_(beta, G_list, H_list, U_list, n)

In [8]:
# Loading test data

with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)

In [9]:
sample_path_jam_test_data_sorted_trans['0']

{u'alphaTrans': 28.0,
 u'startTime': 300.0585,
 u'uuid': [u'8d8c6438-f059-3062-b8f2-d8a948f9c49f',
  u'f48c594a-be63-38d4-87ff-cd316ecbf277']}

In [10]:
i = 0  # index of windows
sam_gap = 100
win_size = 600
win_list = []
while i * sam_gap + win_size < 24 * 3600:
    win_list.append((i * sam_gap, i * sam_gap + win_size))
    i += 1

In [11]:
i

858

In [12]:
win_num = len(win_list)
jam_num = len(sample_path_jam_test_data_sorted_trans)

win_jam = {}
for i in range(win_num):
    key = str(i)
    value = []
    for j in range(jam_num):
        if ((sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] >= win_list[i][0]) \
            and (sample_path_jam_test_data_sorted_trans[str(j)]['startTime'] < win_list[i][1])):
            value.append(str(j))
    win_jam[key] = value

In [35]:
win_jam['857'], win_list[857][0], win_num 

(['1674', '1675', '1676', '1677', '1678', '1679', '1680', '1681'], 85700, 858)

In [64]:
sample_path_jam_test_data_sorted_trans[win_jam['100'][0]]['startTime']

10500.0999

In [14]:
KL_list = []  # The list of the KL divergences; each is for a detection window
etaWC_list = []
for i in range(win_num):
    sample_path_jam = [sample_path_jam_test_data_sorted_trans[key]['alphaTrans'] for key in win_jam[str(i)]]
    n = len(sample_path_jam)
    if (n != 0):
        etaWC_list.append(etaWC(n))
        KL_1 = KL_est(sample_path_jam, mu_1)
        KL_2 = KL_est(sample_path_jam, mu_2)
        KL_3 = KL_est(sample_path_jam, mu_3)
        KL_4 = KL_est(sample_path_jam, mu_4)
        KL_list.append(min([KL_1, KL_2, KL_3, KL_4]))
    else:
        etaWC_list.append(1e-6)
        KL_list.append(0)

In [ ]:
time_range = [sample_path_jam_test_data_sorted_trans[win_jam[str(i)][0]]['startTime'] for i in range(win_num)]

In [66]:
win_jam[str(100)], win_list[100][1]

(['60', '61', '62'], 10600)

In [74]:
time_range = [win_list[i][0] for i in range(win_num)]
plot_points(time_range, KL_list, etaWC_list)
plt.ylabel('divergence')
plt.xlabel('time (s)')
pylab.ylim(-0.1, max(etaWC_list)+0.5)
pylab.xlim(0, max(time_range) + 100)
plt.savefig(data_folder + 'detec_results_WC_robust.pdf')
plt.show()

In [76]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(858, 858, 858, [0, 100])

In [81]:
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [91]:
sample_path_jam_test_data_sorted_trans['532'], sample_path_jam_test_data_sorted_trans['1140']

({u'alphaTrans': 4.0,
  u'startTime': 51900.0063,
  u'uuid': [u'81971278-86c3-34c4-9724-a509f85b0c01',
   u'869012af-1c7e-3d84-9824-940fd2530b28']},
 {u'alphaTrans': 14.0,
  u'startTime': 74700.0083,
  u'uuid': [u'21883f62-022e-3f3c-9405-cc9165112fcd',
   u'9ab64694-243c-318f-a57e-00bcc94c7a1c']})

In [87]:
set(potential_ano_list)

{'1140',
 '1141',
 '1142',
 '1143',
 '1144',
 '1145',
 '1146',
 '1147',
 '1148',
 '1149',
 '1150',
 '1151',
 '1152',
 '1153',
 '1154',
 '1155',
 '1156',
 '1157',
 '1158',
 '1159',
 '1160',
 '1161',
 '1162',
 '1163',
 '1164',
 '1165',
 '1166',
 '1167',
 '1168',
 '1169',
 '1170',
 '1171',
 '1172',
 '1173',
 '1174',
 '1175',
 '1176',
 '1177',
 '1178',
 '1179',
 '1180',
 '1181',
 '1182',
 '1183',
 '1184',
 '1185',
 '1186',
 '1187',
 '1188',
 '1189',
 '1190',
 '1191',
 '1192',
 '1193',
 '1194',
 '1195',
 '1196',
 '1197',
 '1198',
 '1225',
 '1226',
 '1227',
 '1228',
 '1229',
 '1230',
 '1231',
 '1232',
 '1233',
 '1234',
 '1235',
 '1236',
 '1237',
 '1238',
 '1239',
 '1240',
 '1241',
 '1242',
 '1243',
 '1244',
 '1245',
 '1246',
 '1247',
 '1248',
 '1249',
 '1250',
 '1251',
 '1252',
 '1253',
 '1254',
 '1255',
 '1256',
 '1257',
 '1342',
 '1343',
 '1344',
 '1345',
 '1346',
 '1347',
 '1348',
 '1349',
 '1350',
 '1351',
 '1352',
 '1353',
 '1354',
 '1355',
 '1356',
 '1357',
 '1358',
 '1359',
 '1360',
 